### Exercise 1:

Given multiple dictionaries, create a single dictionary to combine these dictionaries and sort it by value, implementing `defaultdict` and `Counter` respectively. 

In [1]:
d1 = {'python': 10, 'java': 3, 'c#': 8, 'js': 15}
d2 = {'java': 10, 'c++': 10, 'c#': 4, 'go': 9, 'python': 6}
d3 = {'erlang': 5, 'haskell': 2, 'python': 1, 'pascal': 1}

In [2]:
from collections import defaultdict, Counter

> Using `defaultdict`

In [3]:
def merge_and_sort(*dicts):
    merged_dict = defaultdict(lambda: 0)
    for d in dicts:
        for k, v in d.items():
            merged_dict[k] += v  
            
    return dict(sorted(merged_dict.items(), key=lambda x:x[1], reverse=True))

In [4]:
merge_and_sort(d1, d2, d3)

{'python': 17,
 'js': 15,
 'java': 13,
 'c#': 12,
 'c++': 10,
 'go': 9,
 'erlang': 5,
 'haskell': 2,
 'pascal': 1}

> Using `Counter`

In [5]:
def merge_and_sort(*dicts):
    merged_dict = Counter()
    for d in dicts:
        merged_dict.update(d)
    
    return dict(merged_dict.most_common())

In [6]:
merge_and_sort(d1, d2, d3)

{'python': 17,
 'js': 15,
 'java': 13,
 'c#': 12,
 'c++': 10,
 'go': 9,
 'erlang': 5,
 'haskell': 2,
 'pascal': 1}

---

### Exercise 2:

Write a function that returns a dictionary with the correct counts for each eye color in the given list.

In [7]:
eye_colors = (
    'amber', 
    'blue',
    'brown',
    'gray',
    'green',
    'hazel',
    'red',
    'violet')

In [8]:
class Person:
    def __init__(self, eye_color):
        self.eye_color = eye_color

In [9]:
from random import seed, choices
seed(56)

persons = [Person(color) for color in choices(eye_colors, k=9)]

In [10]:
def eye_color_counter(persons, eye_colors):
    counter = Counter(p.eye_color for p in persons)
    for c in eye_colors:
        counter.setdefault(c, 0)
    return dict(sorted(counter.items(), key=lambda x:x[0]))

In [11]:
eye_color_counter(persons=persons, eye_colors=eye_colors)

{'amber': 0,
 'blue': 1,
 'brown': 1,
 'gray': 2,
 'green': 2,
 'hazel': 0,
 'red': 1,
 'violet': 2}

---

### Exercise 3:

Given 3 JSON files, representing a default set of settings, and environment specific settings.
* `common.json`
* `dev.json`
* `prod.json`

Write a function that has a single argument (the environment name) and returns the `combined` dictionary that merges the two dictionaries together, with the environment specific settings overriding any common settings already defined.

For simplicity, assume that the argument values are going to be the same as the file names, without the `.json` extension. So for example, `dev` or `prod`.

In [12]:
import json
from pprint import pprint
from collections import ChainMap

In [13]:
def load(env) -> dict:
    with open(f'{env}.json') as f:
        settings = json.load(f)
        return settings

In [14]:
common_settings = load('common')
dev_settings = load('dev')
prod_settings = load('prod')

In [15]:
def chain_recursive(d1, d2):
    chain = ChainMap(d1, d2)
    for k,v in d1.items():
        if isinstance(v, dict) and k in d2:
            chain[k] = chain_recursive(d1[k], d2[k])
    return chain

In [16]:
pprint(chain_recursive(dev_settings, common_settings))

ChainMap({'data': ChainMap({'input_root': '/dev/path/inputs',
                            'numerics': ChainMap({'type': 'float'},
                                                 {'precision': 6,
                                                  'type': 'Decimal'}),
                            'operators': {'add': '__add__'},
                            'output_root': '/dev/path/outputs'},
                           {'input_root': '/default/path/inputs',
                            'numerics': {'precision': 6, 'type': 'Decimal'},
                            'output_root': '/default/path/outputs'}),
          'database': ChainMap({'pwd': 'test', 'user': 'test'},
                               {'db_name': 'deepdive',
                                'port': 5432,
                                'schema': 'public'}),
          'logs': ChainMap({'format': '%(asctime)s: %(levelname)s: '
                                      '%(clientip)s %(user)s %(filename)s '
                              